In [ ]:
import os
import time
import joblib
import numpy as np
import pandas as pd

from tqdm import tqdm
from fastai.vision.all import *
from utils.tissue_mask import get_tissue_mask
from utils.heatmap import crop_img, get_heatmap
from utils.extract_feature_probsmap import get_probsmap_feature

# Need Config

In [ ]:
read_level = 2
down_sample = 256
crop_size = 320
img_size = 320
up_level = 3
infer_bs_size = 32
model = densenet201
model_path = './models/patch_model'
ml_model_path = './models/wsi_model.m'

submit_format_path = '/home/fm/tissuenet/submission/data/submission_format.csv'
test_metadata_csv = '/home/fm/tissuenet/submission/data/test_metadata.csv'
test_tif_base_path = '/home/fm/tissuenet/data/train/tif/'

In [ ]:
def pred_wsi(probs_map, ml_model_path):
    ml_model = joblib.load(ml_model_path)
    x = get_probsmap_feature(probs_map, mode='Test')
    x = np.array(x).reshape(1, -1)
    pred = ml_model.predict(x)
    return pred[0]


def process_one_tif(wsi_path, down_sample, read_level, model_path, 
                    ml_model_path, up_level, crop_size, model, img_size, infer_bs_size):
    tissue_mask = get_tissue_mask(wsi_path, read_level, down_sample, up_level)
    x_start_list, y_start_list, region_df = crop_img(tissue_mask, wsi_path, read_level, down_sample, crop_size)
    probs_map = get_heatmap(wsi_path, tissue_mask, x_start_list, y_start_list, region_df, model_path, model, img_size, infer_bs_size)
    temp_pmp = np.concatenate([np.array(tissue_mask)[np.newaxis, :, :], probs_map], axis=0)
    label = pred_wsi(temp_pmp, ml_model_path)
    return label

In [ ]:
submission_format = pd.read_csv(submit_format_path, index_col=0)
input_metadata = pd.read_csv(test_metadata_csv, index_col=0)
processed_number = 0

for filename, _ in tqdm(input_metadata.iterrows(), total=len(input_metadata)):
    wsi_path_temp = os.path.join(test_tif_base_path, filename)
    pred = process_one_tif(wsi_path_temp, down_sample, read_level, model_path, ml_model_path, 
                           up_level, crop_size, 
                           model, img_size, infer_bs_size)
    submission_format.loc[filename, str(pred)] = 1
    processed_number += 1

    submission_format.to_csv("./submission.csv")